I am just having some fun exploring the dataset and potential features to use for predictions. I decided to focus on adoptions that were only one pet (instead of a whole litter for instance).

In [ ]:
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("../input/train/train.csv")
df.head(5)

In [ ]:
df.info()

In [ ]:
import seaborn as sns
ax = sns.countplot(x="Quantity", data=df)

In [ ]:
#focus on single-pet adoptions only
df_singles = df.loc[df['Quantity'] == 1]
ax = sns.countplot(x="Quantity", data=df_singles)

In [ ]:
df_singles['AdoptionSpeed'].describe()

In [ ]:
ax = sns.countplot(x="AdoptionSpeed", data=df_singles)

In [ ]:
df_singles['Age'].describe()

In [ ]:
df_singles['age_quartile'] = pd.qcut(df_singles['Age'], 4)
ax = sns.boxplot(x="age_quartile", y="AdoptionSpeed", data=df_singles)

In [ ]:
ax = sns.boxplot(x="Gender", y="AdoptionSpeed", data=df_singles)

In [ ]:
ax = sns.boxplot(x="MaturitySize", y="AdoptionSpeed", data=df_singles)

In [ ]:
import numpy as np
df_singles['Medium'] = np.where(df_singles['MaturitySize']==2, 'Yes', 'No')
ax = sns.boxplot(x="Medium", y="AdoptionSpeed", data=df_singles)

In [ ]:
ax = sns.boxplot(x="FurLength", y="AdoptionSpeed", data=df_singles)

In [ ]:
df_singles['ShortHair'] = np.where(df_singles['FurLength']==1, 'Yes', 'No')
ax = sns.boxplot(x="ShortHair", y="AdoptionSpeed", data=df_singles)

In [ ]:
ax = sns.boxplot(x="Health", y="AdoptionSpeed", data=df_singles)

In [ ]:
ax = sns.boxplot(x="Sterilized", y="AdoptionSpeed", data=df_singles)

In [ ]:
ax = sns.countplot(x="Sterilized", hue="age_quartile", data=df_singles)

In [ ]:
ax = sns.boxplot(x="Dewormed", y="AdoptionSpeed", data=df_singles)

In [ ]:
ax = sns.countplot(x="Dewormed", hue="age_quartile", data=df_singles)

In [ ]:
ax = sns.boxplot(x="Vaccinated", y="AdoptionSpeed", data=df_singles)

In [ ]:
ax = sns.countplot(x="Vaccinated", hue="age_quartile", data=df_singles)

In [ ]:
ax = sns.boxplot(x="Type", y="AdoptionSpeed", data=df_singles)

In [ ]:
import numpy as np
df_singles['Free'] = np.where(df_singles['Fee']==50, 'Yes', 'No')
ax = sns.boxplot(x="Free", y="AdoptionSpeed", data=df_singles)

In [ ]:
df_singles['Photos'] = np.where(df_singles['PhotoAmt']==0, 'No', 'Yes')
ax = sns.boxplot(x="Photos", y="AdoptionSpeed", data=df_singles)

In [ ]:
df_singles['Videos'] = np.where(df_singles['VideoAmt']==0, 'No', 'Yes')
ax = sns.boxplot(x="Videos", y="AdoptionSpeed", data=df_singles)

In [ ]:
temp = df_singles.groupby('Color1')['AdoptionSpeed'].mean()
temp.sort_values()

In [ ]:
#in what states are animals adopted the fastest on average?
temp = pd.merge(df_singles, pd.read_csv("../input/state_labels.csv"), left_on='State',right_on='StateID')
temp = temp.groupby('StateName')['AdoptionSpeed'].mean()
temp.sort_values()

In [ ]:
#which breed on average is adopted the fastest? 
temp = pd.merge(df_singles, pd.read_csv("../input/breed_labels.csv"), left_on='Breed1',right_on='BreedID')
temp = temp.groupby('BreedName')['AdoptionSpeed'].mean()
temp.sort_values()

In [ ]:
df_singles['Description']

In [ ]:
df_singles['Guard'] = np.where(df_singles['Description'].str.contains("guard", case=False), '1', '0')
ax = sns.violinplot(x="Guard", y="AdoptionSpeed", data=df_singles)

"Guard" dogs appear to take longer to get adopted. Maybe including that in the description is a turn-off and implies aggressive.

In [ ]:
df_singles['Friend'] = np.where(df_singles['Description'].str.contains("friend", case=False), '1', '0')
ax = sns.violinplot(x="Friend", y="AdoptionSpeed", data=df_singles)

This plot suggests that having the word "friend" or some variaton doesn't really have an impact on the speed of adoption.

Below code inspired  from: https://www.ritchieng.com/machine-learning-k-nearest-neighbors-knn/ 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
X = df_singles[['Age','Gender','Health','FurLength','MaturitySize','Type','Fee',
               'Guard']] #see how these predictors perform
y = df_singles['AdoptionSpeed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

k_range = range(1, 51)
scores = []
features = []

for k in k_range: #trying to find optimal k
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    scores.append(metrics.accuracy_score(y_test, y_pred))

import matplotlib.pyplot as plt
plt.plot(k_range, scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Testing Accuracy')

The best accuracy score is just below 34%. Not great! About one-third of the time the predictions are correct using this model. But we can see that we don't really benefit a whole lot more after using about 30 nearest neighbors.